In [2]:
import numpy as np
from scipy.stats import multivariate_normal as mv
from matplotlib.mlab import bivariate_normal
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import random
import scipy
%matplotlib tk

## Part A

In [10]:
# Reference link: https://stackoverflow.com/a/38705297

# Creating normal functions
mu1 = np.transpose(np.array([0,0]))
mu2 = np.transpose(np.array([5,5]))

sigma = np.array([[0.25,0],[0,2]])
sigma = sigma.diagonal(0)

n1 = mv(mu1,sigma)
n2 = mv(mu2,sigma)

# Creating a grid
x1 = np.linspace(-10,10,500)
x2 = np.linspace(-5,10,500)
X1,X2 = np.meshgrid(x1,x2)
X = np.dstack((X1,X2))
Prob = 0.5*(n1.pdf(X)+n2.pdf(X))

# Plotting
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot_surface(X1,X2,Prob,cmap='Greys')
ax.set_xlabel('X1')
ax.set_ylabel('X2')
ax.set_zlabel('Probability')
plt.show()

## Part B

In [126]:
def probFunc(X):
    # Creating normal functions
    mu1 = np.transpose(np.array([0,0]))
    mu2 = np.transpose(np.array([5,5]))
    
    # 
    sigma = np.array([[0.25,0],[0,2]])
    sigma = sigma.diagonal(0)

    n1 = mv(mu1,sigma)
    n2 = mv(mu2,sigma)

    prob = 0.5*(n1.pdf(X)+n2.pdf(X))
    
    return prob


In [86]:
def mcmc(x_int,steps=10**4,step_size=[[6,0],[0,6]]):
    '''
    MCMC two multivariate normal distributions 
    x_int is the intial values
    steps are the number of steps
    step_size is how large the steps are
    
    Will show the plots of:
    Scatter Plots
    Trace of x1 and x2
    Histogram of x1 and x2
    Histogram of acceptance or rejection
    
    Returns the 2D array of x ([x1,x2]) and the mean
    '''
    # Setting up the walker
    sigma = np.array(step_size)
    sigma = sigma.diagonal(0)
    x = x_int
    x1 = []
    x2 = []
    accept = []
    
    # Doing the burn in using 10% of the steps
    for i in range(int(.1*steps)):
        x_prime = scipy.stats.multivariate_normal(x,sigma).rvs()
        prob = probFunc(x_prime)/probFunc(x)
        if prob > 1:
            x = x_prime 
        else: 
            rand = random.uniform(0,1)    
            if rand <= prob:
                x = x_prime
            else:
                x_prime = 0
    
    # Running the MCMC
    for i in range(steps):
        x_prime = scipy.stats.multivariate_normal(x,sigma).rvs()
        prob = probFunc(x_prime)/probFunc(x)
        
        # If higher prob is higher, accepts
        if prob > 1:
            x = x_prime
            x1.append(x[0])
            x2.append(x[1])
            accept.append(1)
        
        # If less than 1, draws a random int
        else: 
            rand = random.uniform(0,1)    
            if rand <= prob: # accepts
                x = x_prime
                x1.append(x[0])
                x2.append(x[1])
                accept.append(1)
            else: # rejects
                x_prime = 0
                x1.append(x[0])
                x2.append(x[1])
                accept.append(0)
    
    # putting everything into arrays
    x = np.array([x1,x2])
    mean = np.mean(x)

    # Plotting the scatter plot
    s = plt.figure(1)
    plt.scatter(x[0],x[1],color='black',s=.5)
    plt.xlabel('$X_1$')
    plt.ylabel('$X_2$')
    s.show()
    
    # Plotting the trace
    steps = np.linspace(0,steps,steps)
    t, ax = plt.subplots(nrows=2,ncols=1,sharex=True)
    ax[0].plot(steps,x[0],color='black',label='$X_1$')
    ax[1].plot(steps,x[1],color='red',label='$X_2$')
    ax[0].set_ylabel('$X_1$')
    ax[1].set_ylabel('$X_2$')
    t.subplots_adjust(wspace=None)
    t.show()
    
    # Plotting the histogram of x1 and x2
    h = plt.figure(3)
    plt.hist(x[0],color='black',histtype='step',bins=100,label='$X_1$')
    plt.hist(x[1],color='red',histtype='step',bins=100,label='$X_2$')
    plt.legend(loc=0)
    h.show()
    
    # Plotting the acceptance
    h2 = plt.figure(4)
    plt.hist(accept,color='black',histtype='step')
    h2.show()
    
    return x,mean
    
    
    

In [12]:
x,mean = mcmc([10,10],steps=10**5,step_size=[[6,0],[0,6]])
print(mean)

2.5036401925701215


## Part C

In [144]:
# Creating functions to do the stuff
# Found out how to make an arbitrary number of list: 
# https://stackoverflow.com/questions/13520876/how-can-i-make-multiple-empty-lists-in-python

def partition(arr,bins):
    '''
    Breaks up an array into sub-arrays.
    If the array doesn't perfect break up, will just take the remainder at the end
    '''
    # makes list for N number of parameteres
    if len(arr) != 1:
        lists = [[] for _ in range(len(arr))]
    else:
        lists = []
        
    # Loops through each parameter and bins it up
    for i in range(len(lists)):
        # Putting the data into one array
        data = arr[i]
        
        # Placing the new array
        new_arr = lists[i]

        # How many time we can fully bin the array
        iterations = len(data)//bins

        # Breaking up the array
        for j in range(iterations):
            new_arr.append(data[:bins])
            data = data[bins:]
        
        # The remainder of the array is put at the end
        new_arr.append(arr[i])
        
        # Putting it back into the list 
        lists[i] = np.array(new_arr)
        
    return np.array(lists)

def meanError(arr,bins):
    '''
    Takes in an array resulting from an MCMC and caluclates the error associated with it 
    '''
    # This splits the array
    new_arr = partition(arr,bins)
    
    # Makes new arrays for the means
    means = [[] for _ in range(len(new_arr))]
    err = [[] for _ in range(len(new_arr))]
    
    # Takes the mean of each sub array and puts it into the mean array
    # Calculates the error through usual error estiamte: sqrt(variance/N)
    for i in range(len(means)):
        data = np.array(new_arr[i])
        for j in range(len(data)):
            means[i].append(np.mean(data[j]))
        err[i].append(np.sqrt(np.var(np.array(means[i]))/np.array(means[i]).shape[0]))
    err = np.array(err)
    means = np.array(means)
    
    # Calculates total error
    # error = sqrt(sigma_1**2+sigma_2**2+...+sigma_n**2)
    error = []
    for i in range(len(err)):
        error.append(err[i]**2)
    error = np.sqrt(np.sum(np.array(error)))/len(error)
    
    return error,means

    

In [157]:
err = []
mean = []
bins = [10,20,50,100,200,500,1000,2000,5000,10000,20000,30000,40000,50000]

for i in bins:
    result = meanError(x,i)
    err.append(result[0])
    mean.append(result[1])
    
err = np.array(err)

plt.scatter(bins,err,color='k',marker='o',s=5)
plt.show()

In [147]:
err,mean

(0.04752948690293052,
 array([[1.83670807, 5.09914868, 4.98104605, ..., 2.02501354, 2.5341001 ,
         2.50711739],
        [1.8250662 , 5.06221845, 5.36708228, ..., 1.842469  , 2.15238342,
         2.500163  ]]))